In [1]:
import os
import pandas as pd
import numpy as np
import glob
import re
from functools import reduce

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/01_kraken2_mapping/gtdb207_rumenmagsfull/bracken/")

In [3]:
bracken_out = glob.glob("*_s.bracken")

In [4]:
df_list = []
taxaid_dic = {}
for res in bracken_out:
    sample = res.split('_s.bracken')[0]
    df_tmp = pd.read_csv(res, sep='\t')[["name", "taxonomy_id", "fraction_total_reads"]].query('fraction_total_reads > 0.00001')
    df = df_tmp.drop("taxonomy_id", axis = 1)
    df.rename(columns={'name':'species', 'fraction_total_reads':sample}, inplace = True)
    df_list.append(df)
    
    ## get taxaid, taxaid larger than 4294860312 is MAG (from the rumen sample) without official name in GTDB R207
    for index, row in df_tmp.iterrows():
        taxaid_dic_tmp = {}
        taxa = row["name"]
        taxaid = row["taxonomy_id"]
        taxaid_dic[taxa] = taxaid
        
    taxaid_dic.update(taxaid_dic_tmp)

species_profile = reduce(lambda  left,right: pd.merge(left,right,on='species',
                                            how='outer'), df_list).fillna(0)

In [5]:
# add taxaid to the df
taxaid_df = pd.DataFrame.from_dict(taxaid_dic, orient = "index")
taxaid_df.rename(columns={0:"taxaid"}, inplace = True)
species_profile = pd.merge(species_profile, taxaid_df, left_on = "species", right_index = True)
# reorder columns
columns_tmp = [f for f in species_profile.columns.to_list() if f not in ["taxaid", "species"]]
species_profile = species_profile[['species', 'taxaid'] + columns_tmp]

In [6]:
species_profile.set_index('species', inplace = True)

In [7]:
# GTDB R207 taxonomy
bac_gtdb = pd.read_csv("/fs/scratch/PAS0439/Ming/databases/gtdb_R207_repre_genomes/bac120_taxonomy_r207.tsv", sep= "\t", header = None, names=["tmp","taxa"])[["taxa"]]
bac_gtdb = bac_gtdb.taxa.str.split(';', expand = True)
bac_gtdb.rename(columns = {0:"Domain", 1:"Phylum", 2:"Class", 3:"Order", 4:"Family", 5:"Genera", 6:"Species"}, inplace = True)

arc_gtdb = pd.read_csv("/fs/scratch/PAS0439/Ming/databases/gtdb_R207_repre_genomes/ar53_taxonomy_r207.tsv", sep= "\t", header = None, names=["tmp","taxa"])[["taxa"]]
arc_gtdb = arc_gtdb.taxa.str.split(';', expand = True)
arc_gtdb.rename(columns = {0:"Domain", 1:"Phylum", 2:"Class", 3:"Order", 4:"Family", 5:"Genera", 6:"Species"}, inplace = True)

taxa_gtdb = pd.concat([bac_gtdb, arc_gtdb])
## remove suffix before taxa name eg: d__Bacteria -> Bacteria
def rename(x):
    new = re.match(r'[a-z]__(.*)', x).group(1)
    return new

taxa_gtdb = taxa_gtdb.applymap(rename)

In [8]:
# Rumen mags taxonomy, some MAGS remain unclassified in GTDB R207
rumen_mags = pd.read_csv("/fs/scratch/PAS0439/Ming/databases/rumen_mags_high_quality/mags_taxa.tsv", sep="\t")[["user_genome", "Domain", "Phylum", "Class", "Order", "Family", "Genera", "Species"]]
rumen_mags[[f for f in rumen_mags.columns.tolist() if f != "user_genome"]] = rumen_mags[[f for f in rumen_mags.columns.tolist() if f != "user_genome"]].applymap(rename)
rumen_mags = rumen_mags.replace('', "Unclassified")

In [9]:
for index, row in species_profile.iterrows():
    taxaid = row['taxaid']
    if taxaid > 4294860312:
        species_profile.loc[index, "Domain"] = list(rumen_mags.query('user_genome == @index')['Domain'])[0]
        species_profile.loc[index, "Phylum"] = list(rumen_mags.query('user_genome == @index')['Phylum'])[0]
        species_profile.loc[index, "Class"] = list(rumen_mags.query('user_genome == @index')['Class'])[0]
        species_profile.loc[index, "Order"] = list(rumen_mags.query('user_genome == @index')['Order'])[0]
        species_profile.loc[index, "Family"] = list(rumen_mags.query('user_genome == @index')['Family'])[0]
        species_profile.loc[index, "Genera"] = list(rumen_mags.query('user_genome == @index')['Genera'])[0]
    else:
        species_profile.loc[index, "Domain"] = list(set(taxa_gtdb.query('Species == @index')['Domain']))[0]
        species_profile.loc[index, "Phylum"] = list(set(taxa_gtdb.query('Species == @index')['Phylum']))[0]
        species_profile.loc[index, "Class"] = list(set(taxa_gtdb.query('Species == @index')['Class']))[0]
        species_profile.loc[index, "Order"] = list(set(taxa_gtdb.query('Species == @index')['Order']))[0]
        species_profile.loc[index, "Family"] = list(set(taxa_gtdb.query('Species == @index')['Family']))[0]
        species_profile.loc[index, "Genera"] = list(set(taxa_gtdb.query('Species == @index')['Genera']))[0]
        
        

In [10]:
species_profile.to_csv('/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/R_project/microbial_species_profile.tsv', sep = "\t")

In [11]:
## same for genus level

In [12]:
bracken_out_genus = glob.glob("*_g.bracken")

df_list = []
taxaid_dic = {}
for res in bracken_out_genus:
    sample = res.split('_g.bracken')[0]
    df_tmp = pd.read_csv(res, sep='\t')[["name", "taxonomy_id", "fraction_total_reads"]].query('fraction_total_reads > 0.00001')
    df = df_tmp.drop("taxonomy_id", axis = 1)
    df.rename(columns={'name':'genus', 'fraction_total_reads':sample}, inplace = True)
    df_list.append(df)
    
    ## get taxaid, taxaid larger than 4294860313 is MAG (from the rumen sample) without official name in GTDB R207
    for index, row in df_tmp.iterrows():
        taxaid_dic_tmp = {}
        taxa = row["name"]
        taxaid = row["taxonomy_id"]
        taxaid_dic[taxa] = taxaid
        
    taxaid_dic.update(taxaid_dic_tmp)

genus_profile = reduce(lambda  left,right: pd.merge(left,right,on='genus', how='outer'), df_list).fillna(0)


# add taxaid to the df
taxaid_df = pd.DataFrame.from_dict(taxaid_dic, orient = "index")
taxaid_df.rename(columns={0:"taxaid"}, inplace = True)
genus_profile = pd.merge(genus_profile, taxaid_df, left_on = "genus", right_index = True)
# reorder columns
columns_tmp = [f for f in genus_profile.columns.to_list() if f not in ["taxaid", "genus"]]
genus_profile = genus_profile[['genus', 'taxaid'] + columns_tmp]
genus_profile.set_index('genus', inplace = True)


In [13]:
# add higher taxanomic rank 
for index, row in genus_profile.iterrows():
    taxaid = row['taxaid']
    if taxaid > 4294860312:
        genus_profile.loc[index, "Domain"] = list(rumen_mags.query('user_genome == @index')['Domain'])[0]
        genus_profile.loc[index, "Phylum"] = list(rumen_mags.query('user_genome == @index')['Phylum'])[0]
        genus_profile.loc[index, "Class"] = list(rumen_mags.query('user_genome == @index')['Class'])[0]
        genus_profile.loc[index, "Order"] = list(rumen_mags.query('user_genome == @index')['Order'])[0]
        genus_profile.loc[index, "Family"] = list(rumen_mags.query('user_genome == @index')['Family'])[0]
    else:
        genus_profile.loc[index, "Domain"] = list(set(taxa_gtdb.query('Genera == @index')['Domain']))[0]
        genus_profile.loc[index, "Phylum"] = list(set(taxa_gtdb.query('Genera == @index')['Phylum']))[0]
        genus_profile.loc[index, "Class"] = list(set(taxa_gtdb.query('Genera == @index')['Class']))[0]
        genus_profile.loc[index, "Order"] = list(set(taxa_gtdb.query('Genera == @index')['Order']))[0]
        genus_profile.loc[index, "Family"] = list(set(taxa_gtdb.query('Genera == @index')['Family']))[0]






In [14]:
genus_profile.to_csv('/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/R_project/microbial_genus_profile.tsv', sep = "\t")